### Data Initiatialization

In [1]:
from atlassian import Confluence  
import keyring  
import requests
import json
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup as bs 
import re
from jira import JIRA
from jira.client import ResultList
from jira.resources import Issue

from IPython.display import display, HTML, display_html
import pandas as pd
import numpy as np

from dateutil import parser
from datetime import datetime
import datetime as DT

from html import unescape

### Confluence Page Data Scrapper Rules

In [2]:
#get the authentication password from windows credential manager
JIRA_USER = "dapte"
JIRA_URL = "https://jira.opentext.com/"
CONFLUENCE_URL = 'https://confluence.opentext.com/rest/api/content/'

from_date = '2021/07/01' #yyyy/mm/dd
to_date = '2021/07/31' #yyyy/mm/dd

# Todo
confluence_page_id = '516164418' 
#confluence_page_id = '516176304'# - temp page

#Updates Stats - CE
target_version='Target Version stack :'
total_env_updated='Total Environments Updated:'
total_prod_updated = 'Total Production updated:'
update_done_xecmsf = 'xECM for SAP SucessFactors:'
update_done_xecmp ='xECM for SAP Platform:'
update_done_oob ='Office Online Broker:'

#Updates Stats - Complex
update_env_complex = 'Complex Environments Updated:'
update_prod_complex = 'Complex Production updated:'
details_complex = 'Details of Complex Updates:'

# Adhoc Updates
update_done_adhoc = 'Number of Requests Handled:'
details_adhoc = 'Details of Adhoc Updates:'

#Efforts

# Pipeline Updates


### Common Helper Functions

In [3]:
def get_page_json(page_id, expand, username, password, confluence_url):
    url1=confluence_url 
    if expand:
        suffix = "?expand=" + expand 
                              #body.storage
    else:
        suffix = ""

    url = url1 + page_id + suffix
    response = requests.get(url, auth=(username, password))
    response.encoding = "utf8"
    return json.loads(response.text)
    

def set_page_json(page_id,json_content,  username, password, confluence_url):
    headers = {
        'Content-Type': 'application/json',
    }
    url1=confluence_url
    response = requests.put(url1 + page_id, headers=headers, data=json.dumps(json_content),
                 auth=(username, password))
    return(response.text)



### Jira Extractor Functions - Updates

In [4]:
#Updates Stats - CE
def get_target_version():
    return data

def get_jira_count_str(jira,jql,jira_url):
    print(jql)
    issues =pd.Series(jira.search_issues(jql, maxResults=500, fields= 'key', json_result = None),dtype=pd.StringDtype())
    #data = "<a href=\""+str(jira_url)+"\"><b>"+str(issues.size)+"</b></a>"
    #data = {}
    #data.update({'url':str(jira_url),'value':str(issues.size)})
    data = str(issues.size)
    return data

def get_total_env_updated(jira,JIRA_URL,from_date,to_date):
    jql="labels = EMS_Update AND status = Closed  and Resolution = Done  and labels not in ('CAPPU_Documentum_Adhoc_Updates') and component != 'Custom' and duedate  > '"+ str(from_date) +"'   and duedate < '"+str(to_date)+"'"
    url=JIRA_URL+'issues/?jql='+jql
    return get_jira_count_str(jira,jql,url)

def get_total_prod_updated(jira,JIRA_URL,from_date,to_date):
    jql="labels = EMS_Update AND status = Closed  and Resolution = Done  and environment ~ dynamicString('PROD', '')  and duedate  > '"+ str(from_date) +"'   and duedate < '"+str(to_date)+"'"
    url=JIRA_URL+'issues/?jql='+jql
    return get_jira_count_str(jira,jql,url)

def get_emsupdate_xecmsf(jira,JIRA_URL,from_date,to_date):
    jql="labels = EMS_Update AND status = Closed  and Resolution = Done  and labels not in ('CAPPU_Documentum_Adhoc_Updates') and component != 'Custom' and component = 'xECMSF' and duedate  > '"+ str(from_date) +"'   and duedate < '"+str(to_date)+"'"
    url=JIRA_URL+'issues/?jql='+jql
    return get_jira_count_str(jira,jql,url)

def get_update_done_xecmp(jira,JIRA_URL,from_date,to_date):
    jql="labels = EMS_Update AND status = Closed  and Resolution = Done  and labels not in ('CAPPU_Documentum_Adhoc_Updates') and component != 'Custom' and component = 'xECMPF' and duedate  > '"+ str(from_date) +"'   and duedate < '"+str(to_date)+"'"
    url=JIRA_URL+'issues/?jql='+jql
    return get_jira_count_str(jira,jql,url)

def get_update_done_oob(jira,JIRA_URL,from_date,to_date):
    jql="labels = EMS_Update AND status = Closed  and Resolution = Done  and labels not in ('CAPPU_Documentum_Adhoc_Updates') and component != 'Custom' and component = 'OOB' and duedate  > '"+ str(from_date) +"'   and duedate < '"+str(to_date)+"'"
    url=JIRA_URL+'issues/?jql='+jql
    return get_jira_count_str(jira,jql,url)

#Updates Stats - Complex

def get_update_env_complex(jira,JIRA_URL,from_date,to_date):
    jql="labels = EMS_Update AND status = Closed  and Resolution = Done  and labels not in ('CAPPU_Documentum_Adhoc_Updates') and component = 'Custom' and duedate  > '"+ str(from_date) +"'   and duedate < '"+str(to_date)+"'"
    url=JIRA_URL+'issues/?jql='+jql
    return get_jira_count_str(jira,jql,url)

def get_update_prod_complex(jira,JIRA_URL,from_date,to_date):
    jql="labels = EMS_Update AND status = Closed  and Resolution = Done  and labels not in ('CAPPU_Documentum_Adhoc_Updates') and environment ~ dynamicString('PROD', '') and  component = 'Custom' and duedate  > '"+ str(from_date) +"'   and duedate < '"+str(to_date)+"'"
    url=JIRA_URL+'issues/?jql='+jql
    return get_jira_count_str(jira,jql,url)

def get_details_complex():
    return data

# Adhoc Updates
def get_update_done_adhoc(jira,JIRA_URL,from_date,to_date):
    jql=" status = Closed   and component = 'EMS_Adhoc'  and 'duedate'  > '"+ str(from_date) +"'   and 'duedate' < '"+str(to_date)+"'"
    url=JIRA_URL+'issues/?jql='+jql
    return get_jira_count_str(jira,jql,url)
def get_details_adhoc():
    return data



### Confluence functions

In [5]:
def create_update_response(soup,search_str,value):
    newvalue=search_str+'<b>'+value+'</b>'
    tag_li = soup.find_all('li')
    for index,tag in enumerate(tag_li):
        if(search_str in tag.text):
            tag_li[index].clear()
            b = soup.new_tag('b')
            b.string = value
            tag_li[index].insert(0, search_str)
            tag_li[index].insert(1,b)
    return soup

def update_target_version(soup,search_str,value):
    return create_update_response(soup,search_str,value)

def update_total_env_updated(soup,search_str,value_dict):
 #   print(value_dict['url'])
 #   tag_li = soup.find_all('li')
 #   for index,tag in enumerate(tag_li):
 #       if(search_str in tag.text):
 #           print('-----------in if------------')
 #          # print(tag_li[index])
 #           tag_li[index].clear()
 #           a = soup.new_tag('a')
 #           a['href']=value_dict['url']
 #           a.string = value_dict['value']
 #           tag_li[index].insert(0, search_str)
 #           tag_li[index].insert(1,a)

    return  create_update_response(soup,search_str,value_dict)

def update_total_prod_updated(soup,search_str,value):
    return create_update_response(soup,search_str,value)

def publish_count_updated(soup,search_str,value):
    return create_update_response(soup,search_str,value)


### Jira Extractor Fuctions - Provisioning

In [6]:
#todo

### Jira / Confluence Page Iniatializer

In [7]:

keyring_cred=keyring.get_password('keyring_cred', JIRA_USER)

#instantiate a jira object.
jira = JIRA(
    server = JIRA_URL,
    basic_auth=(JIRA_USER, keyring_cred)
)
###
#data = get_total_env_updated(jira,JIRA_URL,from_date,to_date)
#print(data)
#data = get_total_prod_updated(jira,JIRA_URL,from_date,to_date)
#print(data)
#data = get_emsupdate_xecmsf(jira,JIRA_URL,from_date,to_date)
#print(data)
#data = get_update_done_xecmp(jira,JIRA_URL,from_date,to_date)
#print(data)
#data = get_update_done_oob(jira,JIRA_URL,from_date,to_date)
#print(data)
#data = get_update_env_complex(jira,JIRA_URL,from_date,to_date)
#print(data)
#data = get_update_prod_complex(jira,JIRA_URL,from_date,to_date)
#print(data)
#data = get_update_done_adhoc(jira,JIRA_URL,from_date,to_date)
#print(data)
###



#confluence = Confluence(
#    url= CONFLUENCE_URL,
#    username=JIRA_USER, password=keyring_cred
#)


### Data Extraction from Jira

In [8]:
#Todo

### Beautiful Soap Object Updation with Jira Data

In [9]:
#todo

### Validation and Push to Confluence

In [10]:


#516176304
#json_data = get_page_json('516164418',"body.storage")
#json_data = get_page_json('516176304',"body.storage,version", )  get_page_json(page_id, expand, username, password, confluence_url)
json_data = get_page_json(confluence_page_id,"body.storage,version",JIRA_USER,keyring_cred, CONFLUENCE_URL)
page_body = json_data['body']['storage']['value']

soup = BeautifulSoup(page_body, 'html.parser')


#print(page_body)
print('-------------------------------------------------------')
page_body=str(update_target_version(soup,target_version,'21.2'))



page_body= publish_count_updated(soup,total_env_updated,get_total_env_updated(jira,JIRA_URL,from_date,to_date))
page_body =  publish_count_updated(soup,total_prod_updated,get_total_prod_updated(jira,JIRA_URL,from_date,to_date))


page_body =  publish_count_updated(soup,update_done_xecmsf,get_update_done_adhoc(jira,JIRA_URL,from_date,to_date))
page_body =  publish_count_updated(soup,update_done_xecmp,get_update_done_xecmp(jira,JIRA_URL,from_date,to_date))
page_body =  publish_count_updated(soup,update_done_oob,get_update_done_oob(jira,JIRA_URL,from_date,to_date))
#Updates Stats - Complex

page_body =  publish_count_updated(soup,update_env_complex,get_update_env_complex(jira,JIRA_URL,from_date,to_date))
page_body =  publish_count_updated(soup,update_prod_complex,get_update_prod_complex(jira,JIRA_URL,from_date,to_date))

# Adhoc Updates
page_body =  publish_count_updated(soup,update_done_adhoc,get_update_done_adhoc(jira,JIRA_URL,from_date,to_date))
#publish_count_updated


#print(page_body)
page_body=str(page_body)


new_json_data = dict()
new_json_data = json_data
new_json_data['id']=json_data['id']
new_json_data['type'] = json_data['type']
new_json_data['title'] = json_data['title']
new_json_data['type'] = json_data['type']
new_json_data['version'] = {"number": json_data['version']['number'] + 1}
    
new_json_data['body']['storage']['value'] = page_body 

# Call this to push the update to confluence 
set_page_json(confluence_page_id,new_json_data,JIRA_USER,keyring_cred,CONFLUENCE_URL)

-------------------------------------------------------
labels = EMS_Update AND status = Closed  and Resolution = Done  and labels not in ('CAPPU_Documentum_Adhoc_Updates') and component != 'Custom' and duedate  > '2021/07/01'   and duedate < '2021/07/31'
labels = EMS_Update AND status = Closed  and Resolution = Done  and environment ~ dynamicString('PROD', '')  and duedate  > '2021/07/01'   and duedate < '2021/07/31'
 status = Closed   and component = 'EMS_Adhoc'  and 'duedate'  > '2021/07/01'   and 'duedate' < '2021/07/31'
labels = EMS_Update AND status = Closed  and Resolution = Done  and labels not in ('CAPPU_Documentum_Adhoc_Updates') and component != 'Custom' and component = 'xECMPF' and duedate  > '2021/07/01'   and duedate < '2021/07/31'
labels = EMS_Update AND status = Closed  and Resolution = Done  and labels not in ('CAPPU_Documentum_Adhoc_Updates') and component != 'Custom' and component = 'OOB' and duedate  > '2021/07/01'   and duedate < '2021/07/31'
labels = EMS_Update AN

'{"id":"516164418","type":"page","status":"current","title":"EMS Dashboard - August 2021","space":{"id":464748548,"key":"CAPT","name":"Cloud Application Provisioning Updates","type":"global","_links":{"webui":"/display/CAPT","self":"https://confluence.opentext.com/rest/api/space/CAPT"},"_expandable":{"metadata":"","icon":"","description":""}},"history":{"latest":true,"createdBy":{"type":"known","username":"dapte","userKey":"8a82f2295eaf5345015ed2f8591400e3","profilePicture":{"path":"/images/icons/profilepics/default.svg","width":48,"height":48,"isDefault":true},"displayName":"Deepak Apte","_links":{"self":"https://confluence.opentext.com/rest/api/user?key=8a82f2295eaf5345015ed2f8591400e3"},"_expandable":{"status":""}},"createdDate":"2021-08-03T04:46:19.343-04:00","_links":{"self":"https://confluence.opentext.com/rest/api/content/516164418/history"},"_expandable":{"lastUpdated":"","previousVersion":"","contributors":"","nextVersion":""}},"version":{"by":{"type":"known","username":"dapte

### 